In [1]:
from game import DuelGame
from player import DummyPlayer, Aggressive, Defensive, Balanced, Opportunist, Healer, RandomBiased
from data_processor import Tracker
from dataset_repo import DatasetRepository
from essential_types import DataSample

from dotenv import load_dotenv
from typing import List, Tuple, Any, Optional, Dict
from functools import lru_cache
import json
import hashlib
import random
import math
import os

load_dotenv()

CONFIG_VARIABLES: Tuple[Tuple[str, type, Any], ...] = (
    # Basic app config
    ("APP_VERSION", str, "1"),
    
    # Game parameters
    ("MAX_TURNS_PER_GAME", int, 50),
    ("SAMPLES_COUNT_PER_RUN", int, 10000),
    
    # Policy data distribution
    ("DUMMY_PLAYER_POLICIES_DATA_DISTRIBUTION_IN_RUN", list, [0.25, 0.2, 0.15, 0.15, 0.15, 0.1]),
    
    # Aggressive policy
    ("AGGRESSIVE_OPPONENTS", list, ["Defensive", "Healer", "Opportunist", "Balanced", "Aggressive"]),
    ("AGGRESSIVE_DISTRIBUTION", list, [0.25, 0.25, 0.25, 0.15, 0.1]),
    
    # Defensive policy (Note: Fixed case to match .env)
    ("DEFENSIVE_OPPONENTS", list, ["Aggressive", "Opportunist", "Balanced", "RandomBiased", "Defensive"]),
    ("DEFENSIVE_DISTRIBUTION", list, [0.3, 0.2, 0.2, 0.2, 0.1]),
    
    # Balanced policy
    ("BALANCED_OPPONENTS", list, ["Balanced", "Aggressive", "Opportunist", "Defensive", "Healer"]),
    ("BALANCED_DISTRIBUTION", list, [0.3, 0.2, 0.2, 0.2, 0.1]),
    
    # Healer policy
    ("HEALER_OPPONENTS", list, ["Aggressive", "Balanced", "Opportunist", "Defensive", "Healer"]),
    ("HEALER_DISTRIBUTION", list, [0.3, 0.2, 0.2, 0.2, 0.1]),
    
    # Opportunist policy
    ("OPPORTUNIST_OPPONENTS", list, ["Aggressive", "Balanced", "Opportunist", "Defensive"]),
    ("OPPORTUNIST_DISTRIBUTION", list, [0.3, 0.3, 0.2, 0.2]),
    
    # Random-Biased policy
    ("RANDOMBIASED_OPPONENTS", list, ["Aggressive", "Defensive", "Healer", "Balanced", "Opportunist", "RandomBiased"]),
    ("RANDOMBIASED_DISTRIBUTION", list, [0.166, 0.166, 0.166, 0.166, 0.166, 0.166]),
    
    # Aggressive parameters
    ("AGGRESSIVE_EPSILON", float, 0.1),
    ("AGGRESSIVE_ATTACK_BIAS", float, 0.7),
    ("AGGRESSIVE_HEAL_THRESHOLD", float, 40.0),
    ("AGGRESSIVE_ATTACK_THREAT_THRESHOLD", float, 0.6),
    ("AGGRESSIVE_ATTACK_THREAT_HISTORY_LENGTH", int, 5),
    
    # Defensive parameters
    ("DEFENSIVE_EPSILON", float, 0.1),
    ("DEFENSIVE_ATTACK_PROB_OPP_HP_LOW", float, 0.6),
    ("DEFENSIVE_OPP_HP_THRESHOLD", float, 30.0),
    ("DEFENSIVE_HEAL_BIAS", float, 0.6),
    ("DEFENSIVE_DEFENSE_BIAS", float, 0.7),
    ("DEFENSIVE_ATTACK_THREAT_THRESHOLD", float, 0.4),
    ("DEFENSIVE_ATTACK_THREAT_HISTORY_LENGTH", int, 5),
    ("DEFENSIVE_ATTACK_START_TURN_THRESHOLD", int, 5),
    
    # Balanced parameters
    ("BALANCED_EPSILON", float, 0.1),
    ("BALANCED_DOMINATION_MARGIN", float, 30.0),
    ("BALANCED_DESPERATION_MARGIN", float, -20.0),
    
    # Healer parameters
    ("HEALER_EPSILON", float, 0.1),
    ("HEALER_HEAL_THRESHOLD", float, 80.0),
    ("HEALER_HEAL_BIAS", float, 0.8),
    ("HEALER_ATTACK_PROB", float, 0.3),
    
    # Opportunist parameters
    ("OPPORTUNIST_EPSILON", float, 0.1),
    ("OPPORTUNIST_DECISION_THRESHOLD", float, 0.5),
    ("OPPORTUNIST_BASE_TURN_NUMBER", int, 5),
    ("OPPORTUNIST_HEAL_THRESHOLD", float, 35.0),
    ("OPPORTUNIST_HEAL_BIAS", float, 0.85),
    
    # Random-Biased parameters
    ("RANDOM_BIASED_W_ATTACK", float, 0.25),
    ("RANDOM_BIASED_W_DEFENSE", float, 0.25),
    ("RANDOM_BIASED_W_DODGE", float, 0.25),
    ("RANDOM_BIASED_W_HEAL", float, 0.25),
)

# ========================================
# HELPER FUNCTIONS
# ========================================
def _parse_env_string(value: Optional[str], default: str="") -> str:
    """Parse string environment variable."""
    return value if value is not None else default

def _parse_env_int(value: Optional[str], default: int) -> int:
    """Parse integer environment variable."""
    if value is None:
        return default
    try:
        return int(value)
    except (ValueError, TypeError):
        return default

def _parse_env_float(value: Optional[str], default: float) -> float:
    """Parse float environment variable."""
    if value is None:
        return default
    try:
        return float(value)
    except (ValueError, TypeError):
        return default

def _parse_env_list(value: Optional[str], default: list) -> list:
    """Parse list from environment variable.
    Supports both JSON arrays and comma-separated values.
    """
    if value is None:
        return default.copy() if default is not None else []
    
    # Try parsing as JSON first
    try:
        parsed = json.loads(value)
        if isinstance(parsed, list):
            return parsed
    except (json.JSONDecodeError, TypeError):
        pass
    
    # Try comma-separated values
    if ',' in value:
        items = [item.strip() for item in value.split(',') if item.strip()]
        # Try to convert numeric items
        converted_items = []
        for item in items:
            try:
                # Try to convert to float if it looks like a number
                if '.' in item:
                    converted_items.append(float(item))
                else:
                    converted_items.append(int(item))
            except ValueError:
                converted_items.append(item)
        return converted_items
    
    # Single value as list
    return [value.strip()] if value.strip() else []

def _parse_env_bool(value: Optional[str], default: bool) -> bool:
    """Parse boolean environment variable."""
    if value is None:
        return default
    
    value_lower = value.lower().strip()
    true_values = {'true', '1', 'yes', 'on', 'y'}
    false_values = {'false', '0', 'no', 'off', 'n'}
    
    if value_lower in true_values:
        return True
    elif value_lower in false_values:
        return False
    else:
        return default

# ========================================
# TYPE PARSER MAPPING
# ========================================
_TYPE_PARSERS = {
    str: _parse_env_string,
    int: _parse_env_int,
    float: _parse_env_float,
    list: _parse_env_list,
    bool: _parse_env_bool,
}

# ========================================
# MAIN CONFIGURATION LOADER
# ========================================
@lru_cache(maxsize=1)
def get_run_configuration() -> Dict[str, Any]:
    """
    Load and parse all configuration variables with proper types.
    
    Returns:
        Dictionary with configuration variables as keys and parsed values.
    """
    config = {}
    
    for var_name, var_type, default_value in CONFIG_VARIABLES:
        # Get raw value from environment
        raw_value = os.getenv(var_name)
        
        # Get appropriate parser for the type
        parser = _TYPE_PARSERS.get(var_type)
        
        if parser is None:
            # If type not supported, use string as fallback
            config[var_name] = raw_value if raw_value is not None else default_value
        else:
            # Parse with type-specific parser
            config[var_name] = parser(raw_value, default_value)
    
    return config

In [2]:
def play_game_and_return_samples(player_1, player_2, max_turns, seed) -> List[DataSample]:
    game = DuelGame(player_1, player_2, max_turns, random.Random(seed))
    tracker = Tracker(game)
    game.set_tracker(tracker)
    game.play_game()

    created_samples = tracker.get_samples()
    return created_samples

def start_run(
    source_type: str,  # 'env' or 'db'
    template_id: int = None,  # Required if source_type='db'
    label: str = None,
    description: str = None,
    run_label: str = None,
    run_note: str = None,
    seed: int = None
):
    """
    Start a dataset run with specified parameters.
    
    Parameters:
    -----------
    source_type : str
        'env' to load config from environment (.env)
        'db' to load existing config template from database
    template_id : int, optional
        Required when source_type='db', the ID of the config template to load
    label : str, optional
        Config label (for new templates from env)
    description : str, optional
        Config description (for new templates from env)
    run_label : str, optional
        Optional label for this run
    run_note : str, optional
        Optional note for this run
    seed : int, optional
        Random seed for this run. If None, auto-generates
    
    Returns:
    --------
    dict with run_id, template_id, seed, and other relevant info
    """
    # Validate parameters
    if source_type not in ['env', 'db']:
        raise ValueError("source_type must be 'env' or 'db'")
    
    if source_type == 'db' and template_id is None:
        raise ValueError("template_id is required when source_type='db'")
    
    # Instantiate a DatasetRepository class
    config = get_run_configuration()

    db_path = os.getenv('DATABASE_PATH')
    data_repo = DatasetRepository(db_path)
    try:
        
        print(f"Dataset Run Initialization - Loading from {source_type.upper()}")
        
        # ----------------------------
        # LOAD CONFIG FROM ENVIRONMENT
        # ----------------------------
        if source_type == 'env':
            # Hash the config deterministically
            config_json = json.dumps(config, sort_keys=True)
            config_hash = hashlib.sha256(config_json.encode()).hexdigest()
            print(config_hash)
            
            # Check config duplication
            existing_template_id = data_repo.is_config_available_given_hash(config_hash)
            
            if isinstance(existing_template_id, int):
                print(f"Existing config template found (ID={existing_template_id})")
                template_id = existing_template_id
            else:
                print("No existing config found. Creating new template.")
                
                # Create template object
                template = {
                    "app_version": int(config['APP_VERSION']),
                    "label": label or "",
                    "description": description or "",
                    "config_json": config_json,
                }
                
                template_id = data_repo.create_config_template(**template)
                print(f"Created new config template with ID: {template_id}")
        
        # --------------------------------
        # LOAD CONFIG TEMPLATE FROM DB
        # --------------------------------
        elif source_type == 'db':
            try:
                existing_config = data_repo.get_config_template(template_id)['config_json']
                if existing_config['APP_VERSION'] != config['APP_VERSION']:
                    raise ValueError(
                        f"Template version mismatch. Template: {existing_config['APP_VERSION']}, "
                        f"Current: {config['APP_VERSION']}"
                    )
                print(f"Loaded config template {template_id}")
            except Exception as e:
                raise ValueError(f"Failed to load template {template_id}: {str(e)}")
        
        # ----------------------------
        # CREATE DATASET RUN
        # ----------------------------
        # Auto-generate seed if not provided
        if seed is None:
            seed = random.randint(0, 2**32 - 1)
        
        run_id = data_repo.create_run(
            template_id=template_id,
            label=run_label or "",
            samples_count=0,
            note=run_note or "",
            seed=seed
        )
        
        print(
            f"Run created successfully "
            f"(run_id={run_id}, run_label={run_label or 'None'}, "
            f"template_id={template_id}, seed={seed})"
        )
        
        # Prepare run-specific configuration
        samples_count = math.ceil(float(config['SAMPLES_COUNT_PER_RUN']) * 1.05)
        
        # ----------------------------
        # RUN GAME SIMULATIONS
        # ----------------------------
        # Master RNG for this run (deterministic)
        rng = random.Random(seed)
        
        ARCHETYPE_CLASSES = {
            "Aggressive": Aggressive,
            "Defensive": Defensive,
            "Balanced": Balanced,
            "Healer": Healer,
            "Opportunist": Opportunist,
            "RandomBiased": RandomBiased,
        }
        
        ARCHETYPE_ORDER = [
            "Aggressive",
            "Defensive",
            "Balanced",
            "Healer",
            "Opportunist",
            "RandomBiased",
        ]
        
        total_target_samples = samples_count
        all_samples = []
        
        # Policy-level distribution
        policy_distribution = config["DUMMY_PLAYER_POLICIES_DATA_DISTRIBUTION_IN_RUN"]
        
        for policy_name, policy_ratio in zip(ARCHETYPE_ORDER, policy_distribution):
            policy_target = math.ceil(total_target_samples * policy_ratio)
            
            opponents = config[f"{policy_name.upper()}_OPPONENTS"]
            opponent_dist = config[f"{policy_name.upper()}_DISTRIBUTION"]
            
            for opponent_name, opp_ratio in zip(opponents, opponent_dist):
                pair_target = math.ceil(policy_target * opp_ratio)
                pair_collected = 0
                
                while pair_collected < pair_target:
                    remaining = pair_target - pair_collected
                    
                    # Adjust max_turns only for final game if needed
                    max_turns = min(
                        config["MAX_TURNS_PER_GAME"],
                        remaining
                    )
                    
                    # Create players with shared RNG
                    p1 = DummyPlayer(ARCHETYPE_CLASSES[policy_name], rng)
                    p2 = DummyPlayer(ARCHETYPE_CLASSES[opponent_name], rng)
                    
                    game = DuelGame(p1, p2, max_turns, rng)

                    p1.set_game(game)
                    p1.set_opponent(p2)
                    p2.set_game(game)
                    p2.set_opponent(p1)

                    tracker = Tracker(game)
                    game.set_tracker(tracker)

                    game.play_game()

                    samples = tracker.get_samples()
                    
                    all_samples.extend(samples)
                    pair_collected += len(samples)
        
        print(f"Total samples collected: {len(all_samples)}")
        
        # Store samples in database
        data_repo.store_samples(all_samples, run_id)
        data_repo.set_samples_count_for_run(run_id, len(all_samples))
        
        # Return run information
        return {
            "run_id": run_id,
            "template_id": template_id,
            "seed": seed,
            "samples_collected": len(all_samples),
            "run_label": run_label,
            "run_note": run_note
        }
    except Exception as e:
        data_repo.close()
        print('database connection has closed')
        raise e
    


In [3]:
parameters = {
    "source_type" : "env",
    "template_id" : None,
    "label" : "Primary Version first Config Template",
    "description": "",
    "run_label": "second run",
    "run_note": None,
    "seed": None
}

start_run(**parameters)


Dataset Run Initialization - Loading from ENV
8e0e12bf562bb8029e9e40c93ecb3f9e8afd291898c2dff928f4228c460c3277
Existing config template found (ID=1)
Run created successfully (run_id=8, run_label=second run, template_id=1, seed=3866800485)
Total samples collected: 10506
Successfully updated run 8 sample_count to 10506


{'run_id': 8,
 'template_id': 1,
 'seed': 3866800485,
 'samples_collected': 10506,
 'run_label': 'second run',
 'run_note': None}